# Deep learning for brightness profile fitting

http://adsabs.harvard.edu/abs/2018MNRAS.475..894T

The half light radius ($R_{e}$) of a galaxy is the radius at which half of the total light of the system is emitted. This assumes the galaxy has either intrinsic spherical symmetry or is at least circularly symmetric as viewed in the plane of the sky.
<img src="https://upload.wikimedia.org/wikipedia/commons/3/3c/Half_light_radius_simple.svg" style="width: 200px;"/>



# Section 1: Setup

### Import libraries

In [ ]:
import os
#os.environ['KERAS_BACKEND'] = 'theano'
#os.environ['THEANO_FLAGS'] = "device=cuda,force_device=True,floatX=float32"
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import MaxPooling2D, Convolution2D, Conv2D
from keras.layers.noise import GaussianNoise
from keras.models import model_from_json
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle


### SET PATH 
Set the path to the folder containing the simulated input datasets:

In [ ]:
#Where you take the simulated data for train/validation
pathinData = '/home/jovyan/InfieriSchool/DeeLegato_1c/' 
#where do you wanna save the results
pathinModel = '/home/jovyan/InfieriSchool/DeeLegato_1c/'

# Section 1: Load Data

I've saved the data that you will use in this tutorial in numpy format. The data consist of 30,000 stamps of simulated HST/CANDELS galaxies (the design matrix X) and the correspondent half light radius, i.e. the parameter that we aim to predict (stored in the target file Y). 


In [ ]:
X = np.load(pathinData+'StampsSimulated_tutorial.npy')
Y = np.load(pathinData+'ParametersSimulated_tutorial.npy') 


#visualize the data
i = 200
plt.imshow(X[i,0,:,:,],clim=(0,.75))

# Section 2: Preprocess the data

## 1. Scale the features


In [ ]:
#=============================================== 
# Right shape
#===============================================
print ('X.shape= ', X.shape)
X = np.expand_dims(X[:,0,:,:], axis=3)
print ('X.shape= ', X.shape)

print ('Y.shape= ', Y.shape)
Y = Y.reshape(-1,1)
print ('Y.shape= ', Y.shape)

#=============================================== 
# Scale (normalize by mean and by standard deviation)
#===============================================
scaler = preprocessing.StandardScaler().fit(Y)
Y=scaler.transform(Y)


## 2. Split into training and test sets

You should define here your dataset for training and for validation. Let's start taking just 4/5 for the training and 1/5 for the validation. If you want you can play with this division to see if you can improve things further.

In [ ]:
# Spliting in Training and Test datasets
print("Total number of images: ", len(X))
X_train = X[0:len(X)//5*4,:,:,:]
print("Total number of images for the training: ", len(X_train))
X_val = X[len(X)//5*4:,:,:,:]
print("Total number of images for the validation: ", len(X_val))

print("Total number of labels: ", len(Y))

Y_train = Y[0:len(Y)//5*4,0]
print("Total number of labels for the training: ", len(Y_train))

Y_val = Y[len(Y)//5*4:,0]
print("Total number of labels for the validation: ", len(Y_val))

print ('Y_train.shape= ', Y_train.shape)
print ('Y_val.shape= ', Y_val.shape)

## 3. Built a CNN model

In [ ]:
def Build_Model():
    ## PARAMETERS OF THE MODEL
    print ('Building the model')
    dropoutpar = 0.15
    img_rows=128
    img_cols=128
    img_channels=1
    depth=16
    nb_dense = 64  


    # KERAS SEQUENTIAL-MODEL
    model = Sequential()

    model.add(Conv2D(depth, (4, 4),activation='relu', input_shape=(img_rows, img_cols, img_channels), padding="same"))
    model.add(Conv2D(depth, (4, 4),activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropoutpar))

    model.add(GaussianNoise(0.01,input_shape=( img_rows, img_cols,img_channels)))
    model.add(Conv2D(4*depth, (3, 3),activation='relu', padding="same"))
    model.add(Conv2D(4*depth, (3, 3),activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(4*depth, (2, 2),activation='relu', padding="same"))
    model.add(Conv2D(4*depth, (2, 2),activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(2*nb_dense))
    model.add(Dense(nb_dense))
    model.add(Dense(1))
    print(model.summary())
    print("The number of layers in the model is: " ,len(model.layers))
    return model

Build_Model()

## 4. Built a training module

Let's TRAIN the model using SGD + momentum  ===> you can try different loss, optimizer and settings

In [ ]:
def Fit_Model(X_train, X_val,  Y_train, Y_val,  model,verbose):

    lr=0.01
    decay=0
    momentum=0.9
    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    
    model.compile(loss='mean_absolute_error', optimizer=sgd)

    #Do you wanna use data augmentation?
    data_augmentation = False
    #hyperparameters of the training. Try different
    batch_size = 64
    nb_epoch = 5

    if data_augmentation == False:
        print('Not using data augmentation.')
        history = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=nb_epoch,
                  validation_data=(X_val, Y_val),
                  shuffle=True,
                  verbose=verbose)
    if data_augmentation == True:
        print('Using real-time data augmentation.')
        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=True)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)

        datagen.fit(X_train)
        callbacks = [EarlyStopping(monitor='val_loss', patience=10), ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]


        # fit the model on the batches generated by datagen.flow()
        history = model.fit_generator(datagen.flow(X_train, Y_train,
                                batch_size=batch_size),
                                samples_per_epoch=X_train.shape[0],
                                callbacks=callbacks,
                                epochs=nb_epoch,
                                validation_data=(X_val, Y_val),
                                verbose=verbose)

        return model, history

## 5. Train and validate

In [ ]:
#BUILT model
model = Build_Model()

#train model
model, history = Fit_Model(X_train, X_val, Y_train, Y_val, model, verbose=2)

# do you wanna save the model?
saveModel = True
if saveModel == True:
    #save scaler
    scalerfile = pathinModel+'scaler.sav'
    pickle.dump(scaler, open(scalerfile, 'wb'))
    model.save(pathinModel+'model.h5')
    print("Save model to disk")

## 6. Evaluate the performance of your DNN

Plot learning curves, look and the training error, validation error and the generalization error. Do we observe overfitting, underfitting? 

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(history.epoch, history.history['loss'], label='loss')
plt.plot(history.epoch, history.history['val_loss'], label='val_loss')
plt.title('Training performance')
plt.legend()
plt.show()
plt.savefig('foo.png')

print("Best validation loss: %.3f" % (np.min(history.history['val_loss'])))
print("at: %d" % np.argmin(history.history['val_loss']))

## 7. A better metric - $R^2$
The choice of the metric to evaluate the performance of our ML is a fundamental step. The choice of the metric depend on the particular task that we aim to solve.  Here we chose the coefficient of regression $R^2$

In [ ]:
val = scaler.inverse_transform(Y_val)
pred = scaler.inverse_transform(model.predict(X_val))
pred = pred[:,0]
mse=np.mean(np.square(pred-val))
R2 = 1. - mse/np.square(np.std(val))
print ('R2=', R2)


#and now we plot prediction versus validation value
fig = plt.figure(figsize=(12,12))
plt.title('Val sample. $R^2$ = %s' % (R2), size=11)
plt.xlabel('Par', fontsize=13)
plt.ylabel("Par Predicted", fontsize=13)
plt.scatter(val, pred)
plt.savefig('scatter.png')

## 7. What's the best you can get?
Optimize the model as much as you can. Try to modify the model and hyperparameters. Ensure you always keep aside a test set to avoid overfitting.

## 8. Test on real data

Now try yourself to test on real: 

1. Load the real data 
2. Plot some of the images to see how real data looks like
3. Test the model as it is on real data (hint: model.evaluate())
4. Load model and keep training (transfer learning) using part of the real data

In [ ]:
from keras.models import load_model
trained_model=load_model('best_model.h5')